In [1]:
import logging
logging.basicConfig(level=logging.INFO)

import numpy as np
import pickle
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences

H:\anaconda3\envs\nlp\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [3]:
def load_model_and_vocabulary(model_path, vocab_path):
    model = tf.keras.models.load_model(model_path)
    with open(vocab_path, "rb") as file:
        vocabulary = pickle.load(file)
    return model, vocabulary


def preprocess_text(text: str) -> str:
    text = text.replace("\n", " ")
    text = " ".join(text.strip().split())
    return text

def generate_sentence(model, vocabulary, input_sentence, min_words=10, temperature=1):
    reverse_vocabulary = {v: k for k, v in vocabulary.items()}
    input_sentence = preprocess_text(input_sentence)

    word_count = len(input_sentence.split())
    
    while word_count <= min_words:
        tokenized_sentence = [vocabulary.get(token, vocabulary['<OOV>']) for token in input_sentence.split()]
        padded_sentence = pad_sequences([tokenized_sentence], maxlen=n_max, padding='pre', value=vocabulary['<PAD>'])

        predictions = model.predict(padded_sentence, verbose=3)[0]
        predictions[0] = 0
        predictions[1] = 0
        predictions = np.array([np.power(p, 1 / temperature) for p in predictions])
        predictions = predictions / np.sum(predictions)
    

        next_word_idx = np.random.choice(range(len(predictions)), p=predictions)
        next_word = reverse_vocabulary.get(next_word_idx, '<OOV>')

        input_sentence += ' ' + next_word
        
        word_count = len(input_sentence.split())
        if word_count > min_words and input_sentence[-1] not in [".", "!", "?"]:
            min_words += 1

    return input_sentence

In [ ]:
model_path = "../LSTM_LM_model/"
vocab_path = "../vocab.pickle"
n_max = 10
nb_to_generate = 10

model, vocabulary = load_model_and_vocabulary(model_path, vocab_path)

In [6]:
input_sentence = "La France"
for i in range(nb_to_generate):
    completed_sentence = generate_sentence(model, vocabulary, input_sentence)
    logging.info(completed_sentence)

INFO:root:La France aussi française aujourd'hui plusieurs années sont difficiles et vos "Monsieur le "Bonjour, L'épidémie nous nous nous "Monsieur Messieurs-Dames.
INFO:root:La France s'appuie sur un agenda concret sur la transformation numérique français des nerfs "Bonjour, "Bonjour, "Bonjour, des relations poussera débusquant ces deux décennies, ces autorisations.
INFO:root:La France est un État qui va continuer à travailler avec la "Merci des dernières semaines "Mesdames "Mesdames qui "Bonjour, L'épidémie il nous existera Messieurs-Dames.
INFO:root:La France continuera à épuiser le terrain et je veux tout tout "Monsieur le incarnerez dans ce monde et nous "Bonjour, L'épidémie des 1 L'épidémie ces 27 deux mois sont ces deux L'épidémie depuis ces quelques Monter cette dernière française "Bonjour, Rouvrir deux much, much, s'achever.
INFO:root:La France a besoin de demande de nouveaux permettre à tous vos l'Appel, si nous nous "Bonjour, 200 d'entre eux, "Bonsoir aux quinquennats de l'éc